随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用tensorflow2.0推荐的keras接口更方便地实现线性回归的训练。

## 3.3.1 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [1]:
import tensorflow as tf

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal(shape=(num_examples, num_inputs), stddev=1)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += tf.random.normal(labels.shape, stddev=0.01)

## 3.3.2 读取数据

虽然tensorflow2.0对于线性回归可以直接拟合，不用再划分数据集，但我们仍学习一下读取数据的方法

In [2]:
from tensorflow import data as tfdata
batch_size = 10
#将训练集的特征和标签组合
dataset = tfdata.Dataset.from_tensor_slices((features, labels))
# 随机读取小批量
dataset = dataset.shuffle(buffer_size=num_examples)
dataset = dataset.batch(batch_size)
data_iter = iter(dataset)

`shuffle` 的 `buffer_size` 参数应大于等于样本数，`batch` 可以指定 `batch_size` 的分割大小。

In [3]:
for X, y in data_iter:
    print(X, y)
    break

tf.Tensor(
[[ 0.10175886  0.51630604]
 [-0.63255894  0.51894855]
 [-0.5209525  -0.6866028 ]
 [-0.12882712  0.8101947 ]
 [-0.5988874   1.4456608 ]
 [ 0.0166984   0.1508902 ]
 [-0.20909515 -0.9742889 ]
 [ 0.15613756 -0.8808652 ]
 [ 1.550271    0.42602912]
 [-0.45991144  1.0115632 ]], shape=(10, 2), dtype=float32) tf.Tensor(
[ 2.6615367   1.165995    5.489628    1.1831775  -1.9027934   3.725333
  7.0853267   7.499738    5.8493514  -0.15579917], shape=(10,), dtype=float32)


使用`iter(dataset)`的方式，只能遍历数据集一次，是一种比较 tricky 的写法，为了复刻原书表达才这样写。这里也给出一种在[官方文档](https://www.tensorflow.org/guide/eager?hl=zh_cn#computing_gradients)中推荐的写法：

In [4]:
for (batch,(X,y)) in enumerate(dataset):
    print(X,y)
    break

tf.Tensor(
[[ 0.94702125 -0.45456073]
 [ 1.6633339  -1.4213421 ]
 [-0.24382123  0.48107526]
 [-1.452199    0.9503278 ]
 [-0.8667515   0.6838996 ]
 [ 0.7686614  -1.3314059 ]
 [ 0.55235314  0.42988425]
 [-0.01217957  1.189159  ]
 [ 2.5107052  -0.6475883 ]
 [ 1.6255945  -0.5957103 ]], shape=(10, 2), dtype=float32) tf.Tensor(
[ 7.643435   12.371853    2.0805485  -1.9295547   0.14209698 10.256723
  3.8474002   0.12014844 11.442372    9.471927  ], shape=(10,), dtype=float32)


## 3.3.3 定义模型和初始化参数

`Tensorflow 2.0`推荐使用`Keras`定义网络，故使用`Keras`定义网络
我们先定义一个模型变量`model`，它是一个`Sequential`实例。
在`Keras`中，`Sequential`实例可以看作是一个串联各个层的容器。

在构造模型时，我们在该容器中依次添加层。
当给定输入数据时，容器中的每一层将依次推断下一层的输入尺寸。
重要的一点是，在`Keras`中我们无须指定每一层输入的形状。
线性回归，输入层与输出层等效为一层全连接层`keras.layers.Dense()`。

`Keras` 中初始化参数由 `kernel_initializer` 和 `bias_initializer` 选项分别设置权重和偏置的初始化方式。我们从 `tensorflow` 导入 `initializers` 模块，指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。`RandomNormal(stddev=0.01)`指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布。偏差参数默认会初始化为零。

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init
model = keras.Sequential()
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01)))

## 3.3.4 定义损失函数

`Tensoflow`在`losses`模块中提供了各种损失函数和自定义损失函数的基类，并直接使用它的均方误差损失作为模型的损失函数。

In [7]:
from tensorflow import losses
loss = losses.MeanSquaredError()

## 3.3.5 定义优化算法

同样，我们也无须自己实现小批量随机梯度下降算法。`tensorflow.keras.optimizers` 模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。下面我们创建一个用于优化model 所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [8]:
from tensorflow.keras import optimizers
trainer = optimizers.SGD(learning_rate=0.03)

## 3.3.6 训练模型

在使用`Tensorflow`训练模型时，我们通过调用`tensorflow.GradientTape`记录动态图梯度，执行`tape.gradient`获得动态图中各变量梯度。通过 `model.trainable_variables` 找到需要更新的变量，并用 `trainer.apply_gradients` 更新权重，完成一步训练。

In [10]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for (batch, (X,y)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            l = loss(model(X, training=True), y)
            
        grads = tape.gradient(l, model.trainable_variables) # 根据损失函数更新梯度
        trainer.apply_gradients(zip(grads, model.trainable_variables))
    
    l = loss(model(features), labels)
    print("epoch %d,loss: %f " % (epoch,l))

epoch 1,loss: 0.000363 
epoch 2,loss: 0.000097 
epoch 3,loss: 0.000096 


下面我们分别比较学到的模型参数和真实的模型参数。我们可以通过model的`get_weights()`来获得其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [15]:
true_w, model.get_weights()[0]

([2, -3.4],
 array([[ 1.9994994],
        [-3.3990877]], dtype=float32))

In [16]:
true_b, model.get_weights()[1]

(4.2, array([4.1998196], dtype=float32))

## 小结

* 使用`Tensorflow`可以更简洁地实现模型。
* `tensorflow.data`模块提供了有关数据处理的工具，`tensorflow.keras.layers`模块定义了大量神经网络的层，`tensorflow.initializers`模块定义了各种初始化方法，`tensorflow.optimizers`模块提供了模型的各种优化算法。

-----------

> 注：本节除了代码之外与原书基本相同，[原书传送门](

In [ ]:
s